## Primjena alternativnih algoritama pri analizi korisnika kreditnih kartica
Ovaj dokument služi da opiše klasterizaciju korisnika na osnovu nekoliko alternativnih algoritama (pored algoritma K-Means koji je opisan u fajlu kmeans_analysis.ipybn), prikaže i analizira rezultate tih algoritama i uporedi ih sa algoritmom K-Means.

Napomena: Sve analize su urađene analogno analizi K-means algoritma, koristeći po dva grafika koji prikazuju boxplot i srednju vrijednost svakog klastera za svaki atribut.

In [ ]:
from data_preprocessing import load_and_preprocess
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN
from sklearn.cluster import MeanShift
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import Birch
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sb

import warnings
warnings.filterwarnings(action="ignore")

%matplotlib inline

original, preprocessed = load_and_preprocess()

pca = PCA(n_components=5)
reduced_dataframe = pca.fit_transform(preprocessed)

## DBSCAN algoritam
Za ovaj algoritam potrebna su dva parametra:
1. eps - epsilon okolina tačke (vrijednost epsilon izračunata na osnovu računanja udaljenosti svake tačke od njenog najbižeg susjeda)
2. min_samples - minimalan broj susjednih tačaka potrebnih da bi se tačka svrstala u tačke jezgra

In [ ]:
alg = DBSCAN(eps=1, min_samples=10)
label = alg.fit_predict(reduced_dataframe)

fig, ax = plt.subplots()
fig.set_size_inches(15, 9)
ax.set_title("5 clusters according to 5 principle components.")
ax.set_xlabel("PC 0")
ax.set_ylabel("PC 1")
plt.scatter(reduced_dataframe[:, 0], reduced_dataframe[:, 1], c=label.astype(float))
plt.show()

In [ ]:
original['CLUSTER_5'] = label

for c in original:
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(15, 6)
    sb.boxplot(x = 'CLUSTER_5', y = c, data=original, ax=ax1)
    avg = original.groupby('CLUSTER_5')[c].agg('mean')
    avg.plot(ax=ax2, title='mean')
    plt.show()

## DBSCAN - analiza klastera
Ovaj algoritam je pronašao 3 klastera. Na osnovu njihove analize zaključujemo da DBSCAN nije pogodan za korišteni skup podataka, što se može vidjeti i na osnovu grafičkog prikaza klastera.

### Klaster 0
Klaster 0 karakterišu korisnici koji u velikim količinama troše sredstva sa kartice (visoke vrijednosti parametara *PURCHASES*, *INSTALLMENTS_PURCHASES* i *ONE_OFF_PURCHASES*), a u malim količinama uzimaju novac na zajam (jako mala vrijednost parametra *CASH_ADVANCE*, kao i parametara *BALANCE* i *BALANCE_FREQUENCY*).

### Klaster 1
Ovaj klaster čine aktivni korisnici korisnici, koji dosta kupuju, ali i u velikoj količini uzimaju novac na zajam. Vrijednosti paraetara koji su vezani za učestalost aktivnosti su visoke, kao i vrijednosti parametaravezanih za kupovinu (*PURCHASES*), uzimanje novca unapred (*CASH_ADVANCE*) i stanje (*BALANCE*).

### Klaster 2
Klasteru 2 pripadaju pasivni korisnici. Vrijednosti parametara vezanih za kupovinu su veoma niske niske, a takođe i vrijednost parametra *CREDIT_CARD_LIMIT*. Na osnovu nešto viših vrijednosti parametara vezanih za uzimanje novca unapred, može se zaključiti da ovi korisnici više pozajmljuju novac nego što ga troše.

## Agglomerative algoritam
Ovaj algoritam spada u hijerarhijsko klasterovanje (klasteri nastaju spajanjem postojećih klastera i obrazuju strukturu stabla). Algoritmu prosljeđujemo broj klastera koji je određen pomoću PCA, što smo koristili i za K-Means algoritam.

In [ ]:
aggloerative = AgglomerativeClustering(n_clusters=5)
label = aggloerative.fit_predict(reduced_dataframe)

fig, ax = plt.subplots()
fig.set_size_inches(15, 9)
ax.set_title("5 clusters according to 5 principle components.")
ax.set_xlabel("PC 0")
ax.set_ylabel("PC 1")
plt.scatter(reduced_dataframe[:, 0], reduced_dataframe[:, 1], c=label.astype(float))
plt.show()

In [ ]:
original['CLUSTER_5'] = label

for c in original:
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(15, 6)
    sb.boxplot(x = 'CLUSTER_5', y = c, data=original, ax=ax1)
    avg = original.groupby('CLUSTER_5')[c].agg('mean')
    avg.plot(ax=ax2, title='mean')
    plt.show()
    

## Agglomerative algoritam - analiza klastera
Rezultat algoritma daje 5 klastera, za koje se može reći da su veoma slični klasterima dobijenim algoritmom K-Means.

### Klaster 0
Ovaj klaster čine **pasivni korisnici**, kod kojih su vrijednosti većine parametara vezanih za kupovinu, limit kreditne kartice i uzimanje novca unapred jako niske. (slično **Klasteru 0** kod K-Means algoritma)

### Klaster 1
Ovom klasteru pripadaju **neodgovorni korisnici**, koji imaju jako visoke vrijednosti parametara *BALANCE* i *CASH_ADVANCE*, i veliku učestalost uzimanja novca unapred (*BALANCE_FREQUENCY*, *CASH_ADVANCE_FREQUENCY*). (slično **Klasteru 3** kod K-Means algoritma) 

### Klaster 2
Korisnici koji pripadaju ovom klasteru **uzimaju novac unapred (na zajam) i jako malo troše novac** (imaju najnižu vrijednost parametra *MONTHLY_AVERAGE_PURCHASES*, i visoku vrijednost parametra *MONTHLY_AVERAGE_CASH_ADVANCE*). (slično **Klasteru 1** kod K-Means algoritma) 

### Klaster 3
Ovaj klaster čine **frekventni i odgovorni korisnici**, koji imaju vodeće vrijednost parametara vezanih za kupovinu (*PURCHASES*, *INSTALLMENTS_PURCHASES*, *ONE_OFF_PURCHASES*), učestalost kupovine (*PURCHASES_FREQUENCY*), uplaćivanje (*PAYMENTS*), a niske vrijednosti parametara vezanih za pozajmljivanje novca (*CASH_ADVANCE*). (slično **Klasteru 4** kod K-Means algoritma) 

### Klaster 4
Klaster 1 čine **česti i prosječni korisnici**, koji imaju visoke vrijednosti parametara vezanih za učestalost akcija (npr. *BALANCE_FREQUENCY*), nisku vrijednost parametra *CREDIT_LIMIT* i prosječne vrijednosti većine ostalih parametara. (slično **Klasteru 2** kod K-Means algoritma) 


## BIRCH algoritam
Ovaj algoritam takođe spada u hijerarhijsko klasterovanje (Balanced Iterative Reducing and Clustering using Hierarchies) i oslanja se na prethodni algoritam, a njegov značaj je u tome što je prilagođen za klasterovanje velikih skupova podataka. Prosljeđuje se parametar threshold (prag), od kojeg zavisi veličina CF (clustering feature) stabla.

In [ ]:
birch = Birch(threshold=0.3)
label = birch.fit_predict(reduced_dataframe)

fig, ax = plt.subplots()
fig.set_size_inches(15, 9)
ax.set_title("5 clusters according to 5 principle components.")
ax.set_xlabel("PC 0")
ax.set_ylabel("PC 1")
plt.scatter(reduced_dataframe[:, 0], reduced_dataframe[:, 1], c=label.astype(float))
plt.show()


In [ ]:
original['CLUSTER_5'] = label

for c in original:
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(15, 6)
    sb.boxplot(x = 'CLUSTER_5', y = c, data=original, ax=ax1)
    avg = original.groupby('CLUSTER_5')[c].agg('mean')
    avg.plot(ax=ax2, title='mean')
    plt.show()
    

## BIRCH algoritam - analiza klastera
Pod pretpostavkom da bi ovaj algoritam dao slične rezultate kao prethodni, umjesto 5 klastera sada tražimo 3 klastera (default-na vrijednost broja klastera ove funkcije) radi upoređianja rezultata.

### Klaster 0
Klaster 0 čine **odgovorni i česti korisnici**, koji imaju visoke vrijednosti parametara *PURCHASES*, *PAYMENT_TO_MIN_PAYMENT_RATIO*, i niske vrijednosti parametara *MONTHLY_AVERAGE_CASH_ADVANCE*, *BALANCE* itd.

### Klaster 1
Klaster 1 čine **neodgovorni i česti korisnici**, sa visokim vrijednostima parametara *PURCHASES*, *CREDIT_LIMIT*, ali i *CASH_ADVANCE*, *BALANCE_TO_CREDIT_LIMIT_RATIO* itd.

### Klaster 2
U klaster 2 spadaju **neodgovorni i rijetki korisnici**, kod kojih su parametri vezani za učestalost i kupovinu jako niski (*PURCHASES*), a parametri vezani za pozajmljivanje novca visoki (*CASH_ADVANCE*).